In [1]:
import tensorflow as tf
import os
import numpy as np

## Load Data

In [2]:
# constants

UNITS = 256

BUFFER_SIZE = 120
BATCH_SIZE = 64

max_name_features = 10000
max_name_len = 4

max_ingredient_features = 4000
max_ingredient_len = 4

max_step_features = 10000
max_step_len = 400

In [3]:
text_path = os.path.join('data', 'choc_recipes.txt')
# Using readlines()
file1 = open(text_path, 'r')
Lines = file1.readlines()

names = []
ingredients = []
steps = []

count = 0
# Strips the newline character
for line in Lines:
    pieces = line.split('\t')
    names.append(pieces[0])
    ingredients.append(pieces[1])
    steps.append(pieces[2])

In [4]:
ingredients_list_list = []
for i_string in ingredients:
    i = i_string.replace(', ', '*')
    i = i.replace(' ', '_')
    i = i.replace('*', ', ')

    ingredients_list_list.append(i)

ingredients = ingredients_list_list

In [5]:
print(f'Name count: {len(names)}')
print(f'Ingredient count: {len(ingredients)}')
print(f'Step count: {len(steps)}')

Name count: 17258
Ingredient count: 17258
Step count: 17258


In [6]:
names = np.array(names)
ingredients = np.array(ingredients)
steps = np.array(steps)

In [7]:
BUFFER_SIZE = len(names)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(names),)) < 0.8

# train
train_dataset = tf.data.Dataset.from_tensor_slices(({"names": names[is_train], "ingredients": ingredients[is_train]}, steps[is_train]))
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

# test
test_dataset = tf.data.Dataset.from_tensor_slices(({"names": names[~is_train], "ingredients": ingredients[~is_train]}, steps[~is_train]))
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

### Create Text Vectorizer

In [8]:
step_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_step_features,
 output_mode='int',
 output_sequence_length=max_step_len)

#train_raw.map(lambda context, target: context)
step_vectorizer.adapt(steps)

In [9]:
step_vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'step', 'and', 'the', 'in', 'a', 'to', 'until', 'with']

In [10]:
ingredient_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_ingredient_features,
 output_mode='int',
 output_sequence_length=max_ingredient_len)

#train_raw.map(lambda context, target: context)

ingredient_vectorizer.adapt(ingredients)

In [11]:
ingredient_vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 'sugar',
 'butter',
 'salt',
 'eggs',
 'flour',
 'vanilla',
 'bakingpowder',
 'milk']

In [12]:
name_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_name_features,
 output_mode='int',
 output_sequence_length=max_name_len)

#train_raw.map(lambda context, target: context)
name_vectorizer.adapt(names)

In [13]:
name_vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 'cake',
 'cookies',
 'chocolate',
 's',
 'pie',
 'cream',
 'apple',
 'with']

In [14]:
def process_text(inputs, outputs):

  names = inputs['names']
  ingredients = inputs['ingredients']

  name_context = name_vectorizer(names)
  ingredient_context = ingredient_vectorizer(ingredients)
  context = {'names': name_context, 'ingredients': ingredient_context}

  target = step_vectorizer(outputs)
  targ_in = target[:,:-1]
  targ_out = target[:,1:]
  return (context, targ_in), targ_out


train_ds = train_dataset.map(process_text, tf.data.AUTOTUNE)
test_ds = test_dataset.map(process_text, tf.data.AUTOTUNE)

## Load Model

In [15]:
model = tf.saved_model.load('translator')

KeyboardInterrupt: 

In [ ]:
result = model.translate(['sugar, salt, eggs, vanilla'], ['cake'])
result[0].numpy().decode()

'cake mix cake mix eggs and oil in a large bowl step beat in eggs one at a time beating well after each addition step add flour mixture and beat until combined step pour into prepared pan step bake at 350 degrees for 25 minutes or until cake tests done'

## Train Model

In [ ]:
history = model.fit(
    train_ds.repeat(), 
    epochs=10,
    steps_per_epoch = 100,
    validation_data=test_ds,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

AttributeError: '_UserObject' object has no attribute 'fit'